Copyright Preferred Computational Chemistry, Inc. as contributors to Matlantis contrib project

# Persistent homology analysis using HomCloud.

**Persistent homology (PH)** is a tool from topological data analysis. PH is used to characterize the complex structural features of point clouds withoud clear periodicity, such as the atomic positions of amorphous solids, porous materials and polymers. In PH analysis, rings (2D) and cages (3D) can be identified with increasing the sphare radii of points. PH tracks the scale at which these features appear (are "born") and disappear (are "die"). These birth-death events are represented in a Persistent Diagram (PD). The PD can then be analyzed to gain chemical and physical insites into the structural properties.
In this example, `HomCloud`, an python package for PH calculations, will be installed on Matlantis.

## References
* https://homcloud.dev/index.en.html
* I. Obayashi, J. Phys. Soc. Jpn. 91, 091013 (2022) https://journals.jps.jp/doi/10.7566/JPSJ.91.091013

## Install HomCloud

Before installing `HomCloud`, please download and install the necessary packages `GNU MPFR` and `CGAL`.

1. Download the latest source archive `mpfr-4.2.X.tar.xz` of `GNU MPFR` from [the official website](https://www.mpfr.org/) and upload to your home directory on Matlantis.

2. Run the cell below. It will download the source archive `CGAL-v5.6.X` with *wget* and build `GNU MPFR` and `CGAL`. 

3. Install `HomCloud` with *pip*.

GNU MPFR https://www.mpfr.org/ \
Dlownload: https://www.mpfr.org/mpfr-current/#download

CGAL https://www.cgal.org \
Github repositry: https://github.com/CGAL/cgal

In [ ]:
cgal_version = '5.6.1'
mpfr_version = '4.2.1'

In [ ]:
%%time
import os
from subprocess import run, PIPE

lines = f"""
wget https://github.com/CGAL/cgal/releases/download/v{cgal_version}/CGAL-{cgal_version}.tar.xz

mkdir -p ~/cgal_install
tar Jxfv CGAL-{cgal_version}.tar.xz
cd CGAL-{cgal_version}
mkdir -p build
cd build
cmake -DCMAKE_INSTALL_PREFIX=~/cgal_install ..
make
make install

cd 

tar Jxfv mpfr-{mpfr_version}.tar.xz
cd mpfr-{mpfr_version}
./configure --prefix=$HOME/mpfr_install
make
make install

export LD_LIBRARY_PATH=/home/jovyan/mpfr_install/lib:/home/jovyan/cgal_install/lib:/home/jovyan/mpfr_install/lib
export CFLAGS="-I$HOME/mpfr_install/include -I$HOME/cgal_install/include -L$HOME/mpfr_install/lib"
"""
with open('install_homcloud.sh', 'w') as f:
    f.write(lines)
cmd = 'bash install_homcloud.sh'.split()
p = run(cmd)

In [ ]:
!pip3 install 'homcloud==4.2.1'

In [ ]:
! pip install  homcloud

## Make a random point cloud.

In this example, a random point cloud is used for the input data. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Generate random points in 3D.
rng = np.random.default_rng() 
pointcloud = rng.random((1000,3))

# Display the points.
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

x, y, z = pointcloud.T
ax.scatter(x, y, z)

## Calculate the persistent diagram using HomCloud.

In [ ]:
%%time
import homcloud
import homcloud.interface as hc

# Calculate the persistent homology.
print(f'HomCloud version : {homcloud.version.__version__}')
hc.PDList.from_alpha_filtration(pointcloud, 
                                save_to="pointcloud.pdgm",
                                save_boundary_map=True)

In [ ]:
pdlist = hc.PDList("pointcloud.pdgm")
pd1 = pdlist.dth_diagram(1)
pd1.histogram().plot()